In [7]:
import tkinter as tk
from tkinter import *
import cv2
from PIL import Image, ImageTk
import os
import numpy as np
import cv2
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from keras.optimizers import Adam
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator


In [8]:
emotion_model = Sequential()
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))
emotion_model.load_weights('face_predict.h5')


In [9]:
import cv2
import numpy as np
from PIL import Image, ImageTk
import tkinter as tk

emotion_dict = {0: "   Angry   ", 1: "Disgusted", 2: "  Fearful  ", 3: "   Happy   ", 4: "  Natural  ", 5: "    Sad    ", 6: "Surprised"}

emoji_dist = {
    0: "images/angry.png",
    1: "images/disgusted.png",
    2: "images/fearful.png",
    3: "images/happy.png",
    4: "images/neutral.png",
    5: "images/sad.png",
    6: "images/surpriced.png"
}


cv2.ocl.setUseOpenCL(False)

# Define dictionaries and classes here...
class ImageDisplayGUI:
    def __init__(self):
        self.root = tk.Tk()
        self.frame = tk.Frame(self.root)
        self.frame.pack()

        self.camera_label = tk.Label(self.frame)
        self.camera_label.pack(side=tk.LEFT, padx=10)

        self.emoji_label = tk.Label(self.frame)
        self.emoji_label.pack(side=tk.RIGHT, padx=10)

        self.text_label = tk.Label(self.root, font=("Helvetica", 16))
        self.text_label.pack(side=tk.TOP, pady=10)

        self.cap = cv2.VideoCapture(0)
        self.show_next_image()

    def show_next_image(self):
        ret, frame = self.cap.read()
        if not ret:
            return
        bounding_box = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        num_faces = bounding_box.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in num_faces:
            cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
            roi_gray_frame = gray_frame[y:y + h, x:x + w]
            cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
            emotion_prediction = emotion_model.predict(cropped_img)
            maxindex = int(np.argmax(emotion_prediction))
            label_text = emotion_dict[maxindex]
            
            self.text_label.configure(text=label_text)

            # Update the camera label
            camera_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            camera_image = Image.fromarray(camera_image)
            camera_photo = ImageTk.PhotoImage(camera_image)
            self.camera_label.configure(image=camera_photo)
            self.camera_label.image = camera_photo

            # Update the emoji label
            emoji_image_path = emoji_dist[maxindex]
            emoji_image = Image.open(emoji_image_path)
            emoji_photo = ImageTk.PhotoImage(emoji_image)
            self.emoji_label.configure(image=emoji_photo)
            self.emoji_label.image = emoji_photo

            self.root.after(1500, self.show_next_image)
            return

        self.root.after(1500, self.show_next_image)


    def run(self):
        self.root.mainloop()

gui = ImageDisplayGUI()
gui.run()


1/1 [==============================] - 0s 18ms/step
